In [2]:
import numpy as np
import matplotlib.pyplot as plt
import random as rand

In [54]:
def TableGenerator_withEve(MesString,AliBasString,BobBasString,MeasString,EveBasString,EveMeasString):
    #Intput: 6 arrays of 02 or 1s of same length
    #Basis nomenclature: 0: Z Basis; 1: X Basis; -1 (or something else):NOT measured (only for Eve case)
    #Output: Merge them into a table
    
    Table = np.zeros([6,len(MesString)])
    Table[0,:] = MesString
    Table[1,:] = AliBasString
    Table[2,:] = BobBasString
    Table[3,:] = MeasString
    Table[4,:] = EveBasString
    Table[5,:] = EveMeasString
    
    return Table

def TableShuffler_withEve(Table):
    #Input: Table, a 6xn_iter array
    #Output: Table Shuffled in columns (permutation)
    rng = np.random.default_rng()
    Shuffled = np.transpose(rng.permutation(np.transpose(Table)))
    return Shuffled

def TableFiltering_withEve(Table):
    #Input Table, a 6xn_iter array
    #Output: Table Filtered: Only those columns in which
    #Alice Basis = Bob Basis
    #i = 0: Alice Message
    #i = 1: Alice Basis
    #i = 2: Bob Basis
    #i = 3: Bob Measures
    #i = 4: Eve Basis
    #i = 5: Eve Measures
    
    SameBasisTable = np.zeros([6,1])
    for i in range(len(Table[0,:])):
        if Table[1,i] == Table[2,i]:
            newcol = np.zeros([6,1])
            for j in range(6):
                newcol[j,0]=Table[j,i]
            SameBasisTable = np.append(SameBasisTable,newcol,axis=1)
    SameBasisTable = np.delete(SameBasisTable,0,axis=1)
    return SameBasisTable

def TableSampling_withEve(Table,frac=0.1):
    #Extracts a sample to caclulate the raw probability of failure and expose Eve, if present
    #Input: Table to work upon, fraction of the columns devoted to sampling
    random_indx = np.random.randint(0,len(Table[0,:]),int(frac*len(Table[0,:])))
    SampleTable = Table[:,random_indx]
    
    sum = 0
    for i in range(len(SampleTable[0,:])):
        if SampleTable[0,i]==SampleTable[3,i]:
            sum +=1
    
    rawSuccessRate = sum/float(len(SampleTable[0,:]))
    
    newTable = Table
    newTable = np.delete(Table,random_indx,axis=1)
    
    return newTable, SampleTable, rawSuccessRate

def TableReorganisation_withEve(Table,n_rep):
    #Input: Filtered Table, number of iterations for repetition code
    #Output: Reorganised table by columns in groups of n_rep
    newTable = Table
    index_row = np.zeros(len(Table[0,:]),dtype = int)
    index_now = 0
    index_list = []
    for i in range(len(Table[0,:])):
        #print(i,"/",len(Table[0,:]))
        #print(index_row)
        if index_row[i] != 0:
            continue
        else:
            index_row[i] = index_now
            index_list.append(i)
            column_counter = 1
            leap = 1
            while column_counter < n_rep and i + leap < len(Table[0,:]): #We have not found all the columns or we have run out of columns
                if Table[0,i]==Table[0,i+leap] and Table[1,i]==Table[1,i+leap] and Table[1,i]==Table[1,i+leap]:
                    index_now += 1
                    column_counter += 1
                    index_row[i+leap] = index_now
                    index_list.append(i+leap)
                leap += 1
            index_now +=1
    
    #Table Reorganisation
    newTable[:] = Table[:,index_list]
    
    
    return newTable

def TableMajorityVoting_withEve(Table,n_rep):
    #Given the reorganised table, merges the columns applying majority voting
    newTable = np.zeros([6,1])
    i = 0
    leap = 0
    while i + leap < len(Table[0,:]):
        newcol = np.zeros([6,1])
        for j in range(3):
            newcol[j,0]=Table[j,i]
        newcol[4,0]=Table[4,0] #Eve Basis string doesn't change
        
        leap = 0
        cache = []
        cache_eve = []
        
        while i + leap < len(Table[0,:]) and Table[0,i]==Table[0,i+leap] and Table[1,i]==Table[1,i+leap] and  leap <n_rep:
            cache.append(Table[3,i+leap])
            
            if Table[4,i+leap]==Table[1,i+leap]: #Alice Basis and Eve basis were the same
                cache_eve.append(Table[5,i+leap])
            
            
            leap += 1
        
        
        if len(cache) == n_rep:
            
            #Majority voting for Bob
            if cache.count(0) > cache.count(1):
                newcol[3,0] = 0
            else:
                newcol[3,0]= 1
            
            #Majority voting for Eve
            if cache_eve.count(0)> cache_eve.count(1):
                newcol[5,0] = 0
            elif cache_eve.count(0)<cache_eve.count(1):
                newcol[5,0] = 1
            else:
                newcol[5,0] = np.random.randint(2) #Flips a coin (Very very bad for Eve!, also very very unlikely)
        
            newTable = np.append(newTable,newcol,axis=1)
        
        
        i = i+leap
    
    newTable = np.delete(newTable,0,axis=1)
    return newTable

def NewStrings_withEve(Table):
    #Given the Mayority Voted Table, returns the new Alice Message, Bob and Eve Measures strings
    newAliceString = Table[0,:]
    newBobString = Table[3,:]
    newEveString = Table[5,:]
    
    return newAliceString,newBobString,newEveString


def WholeRepetitionProcedure_withEve(AliceMess,AliceBas,BobBas,BobMeasure,EveBasis,EveMeasures,n_rep):
    #Input: The 6 strings obtained through BB84 or E91 protocols, and how many repetition bits n_rep
    #Basis nomenclature: 0: Z Basis; 1: X Basis; -1: NOT measured (only for Eve case)
    #Eve Measure string: 0s or 1s if she has measured, -1 if she has not
    #Output: The new AliceKey, BobKey and EveKey after the repetition protocol
    
    Table = TableGenerator_withEve(AliceMess,AlicBas,BobBas,BobMeasure,EveBasis,EveMeasures)
    Shuffled = TableShuffler_withEve(Table)
    Filtered = TableFiltering_withEve(Shuffled)
    Reorganised = TableReorganisation_withEve(Filtered,n_rep)
    Majority = TableMajorityVoting_withEve(Reorganised,n_rep)
    NewAliceKey, NewBobKey, NewEveKey = NewStrings_withEve(Majority)
    
    return NewAliceKey,NewBobKey,NewEveKey
        

In [60]:
#Trial
Gross_length = 30
n_rep = 3
Message = np.random.randint(2,size = Gross_length)
AlicBas = np.random.randint(2,size = Gross_length)
BobBas  = np.random.randint(2,size = Gross_length)
BobMeasure = np.random.randint(2,size = Gross_length)
EveBasis = np.random.randint(2,size = Gross_length)
EveMeasure = np.random.randint(2,size = Gross_length)

Table = TableGenerator_withEve(Message,AlicBas,BobBas,BobMeasure,EveBasis,EveMeasure)
print("Initial Table")
print(Table)
print()

Shuffled = TableShuffler_withEve(Table)
print("Shuffled")
print(Shuffled)
print()

Filtered = TableFiltering_withEve(Shuffled)
print("Filtered")
print(Filtered)
print()

Reorganised = TableReorganisation_withEve(Filtered,n_rep)
print("Reorganised")
print(Reorganised)
print()

Majority = TableMajorityVoting_withEve(Reorganised,n_rep)
print("Majority")
print(Majority)
print()

NewAliceKey, NewBobKey, NewEveKey = NewStrings_withEve(Majority)
print("New Keys")
print("Alice Key:", NewAliceKey)
print("Bob Key: ",NewBobKey)
print("Eve Key: ",NewEveKey)
print()

print("WholeProcedure")
FinalAliceKey, FinalBobKey, FinalEveKey = WholeRepetitionProcedure_withEve(Message,AlicBas,BobBas,BobMeasure,EveBasis,EveMeasure,n_rep)
print("Alice Key:", FinalAliceKey)
print("Bob Key: ",FinalBobKey)
print("Eve Key: ",FinalEveKey)
print()


Initial Table
[[1. 1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 1. 0. 1.
  1. 1. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1.
  1. 0. 1. 1. 1. 1.]
 [1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 0. 1. 0.
  1. 1. 1. 0. 0. 1.]
 [0. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1.
  1. 1. 0. 1. 0. 1.]
 [0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0.
  0. 1. 1. 1. 1. 1.]
 [0. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 0. 1. 1. 1. 0.
  0. 1. 1. 0. 0. 1.]]

Shuffled
[[0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 1. 1. 1. 1.
  0. 1. 0. 0. 0. 1.]
 [0. 1. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 1. 1. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0.
  1. 1. 1. 0. 0. 1.]
 [1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1.
  1. 0. 1. 0. 0. 0.]
 [0. 0. 1. 1. 1. 1. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1.
  0. 1. 1. 0. 1. 0.]
 [1. 1. 0. 1. 1. 0. 1. 1.